In [2]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats
plt.style.use("ggplot")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
#plt.rcParams['font.family'] = 'IPAexGothic'

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle

f = open("mean_vec_list.binaryfile",'rb')
mean_vec_list = pickle.load(f)

In [3]:
import gensim

model = gensim.models.Word2Vec.load("latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [4]:
# 入力単語

imput_word_list = ['コーヒー', "スイーツ", "アイスクリーム", "有名", "店舗"]

In [5]:
result_vec_list  = []

for w in imput_word_list:

    w_to_v = model.wv[w]

    for mean_vec in mean_vec_list:
        result_vec_plus = w_to_v - mean_vec 
        result_vec_minus =   w_to_v + mean_vec 
    
        result_vec_list.append(result_vec_plus)
        result_vec_list.append(result_vec_minus)

In [6]:
len(result_vec_list)

8940

# 計算方法

1. 入力単語に対して, 関係ベクトルを足し算and引き算=>推定用のベクトルを算出
2. 推定ベクトルと単語ベクトルで類似度が高いやつをそれぞれ3個出す
2. 出力単語の頻度
2. 

In [7]:
# 関係ベクトルと類似度が高い単語をn個上位表示

import numpy as np

output_word_list = []
max_n = 2

for vec in result_vec_list:
    most_similar = np.array(list(model.wv.most_similar([vec], [], max_n)))
    
    output_word_list.extend(most_similar[:,0].tolist())

In [8]:
#インプットした単語と同じ単語は除去

for imput_word in imput_word_list:
    output_word_list = [output_word for output_word in output_word_list if output_word != imput_word]
    print(len(output_word_list))

17497
17195
16959
16137
15428


In [9]:
len(output_word_list)

15428

In [10]:
# 登録済固有名詞リスト

f = open("koyuu_noun_list.binaryfile",'rb')
noun_list = pickle.load(f)

In [11]:
len(noun_list)

90128

In [12]:
# 固有名詞リスト外の単語は除去

output_word_list = [output_word for output_word in output_word_list if noun_list.count(output_word)>=1]

In [13]:
len(output_word_list)

3052

In [14]:
len(set(output_word_list))

1141

# 単純なword2vecのmost_similar関数との比較

In [20]:
# 複数の単語で類似度の高いものを上から１０個表示する
most_similar = np.array(list(model.wv.most_similar(imput_word_list, [], 10)))[:,0]
most_similar.tolist()

['ピザ',
 'ハンバーガー',
 'ハンバーグ',
 'シーフード',
 'ステーキ',
 'カレーライス',
 'ソフトクリーム',
 'デザート',
 'お菓子',
 '軽食']

In [21]:
#共起単語と共起回数をdic化してまとめる

import collections

c = collections.Counter(output_word_list)
c

Counter({'酒': 64,
         'スコーン': 3,
         'ムア': 1,
         'ボルドー': 3,
         '夢前川': 3,
         '川舟': 1,
         'ウォッカ': 21,
         'ニューヨーク・タイムズ': 13,
         'ロッシーニ': 1,
         'フィッシュ・アンド・チップス': 15,
         'シュトゥルーデル': 28,
         'フォルジャーズ': 5,
         'エンパナーダ': 1,
         'カレー': 31,
         'ヘレス': 3,
         'ゲータレード': 7,
         'フィレオフィッシュ': 1,
         'コカコーラ': 6,
         'サンドウィッチ': 7,
         'プライベーター': 2,
         'ガンボ': 1,
         'ナンシー': 2,
         'ベジタリアン': 39,
         '漬': 2,
         '麹': 10,
         'カルバン・クライン': 1,
         'カルピス': 2,
         '茶': 40,
         '京阪': 3,
         'バジル': 3,
         'グラッサー': 1,
         'ウエイター': 5,
         'ペストリー': 4,
         'ベーグル': 27,
         'デント': 2,
         'コカ・コーラ': 15,
         'ハリウッド': 4,
         '塩釜': 7,
         'マックイーン': 6,
         'プランテーション': 12,
         'マドレーヌ': 1,
         '芝': 2,
         'デンゼル・ワシントン': 19,
         'フォッシー': 3,
         'シェリー': 28,
         'プルーン': 1,
         'フライドチキン': 23,
   

In [17]:
values = sorted(c.values(), reverse=True)[:50]

In [18]:
nun_list = []

for i in values:

    keys = [k for k, v in c.items() if v == i]
    nun_list.extend(keys)

In [61]:
import numpy as np

def cal_cos_similarity(q, d):
    return np.dot(q, d) / (np.linalg.norm(q) * np.linalg.norm(d))


cal_cos_similarity(model.wv["のび太"], model.wv["鬼塚"])

0.40635827